In [1]:
import json

API_KEY_FILE = 'api_key.txt'
REGION = 'euw1'
BASE_URL = 'api.riotgames.com'


with open(API_KEY_FILE) as f:
    api_key = f.read()
    
def build_url(endpoint):
    return f'https://{REGION}.{BASE_URL}{url}'

def save_json(obj, file):
    with open(f'outputs/{file}', 'w+') as f:
        json.dump(obj, f, indent=4)

In [2]:
import requests

url = "/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5"

payload={}
headers = {
  'X-Riot-Token': api_key
}

response = requests.request("GET", build_url(url), headers=headers, data=payload)
if response.status_code == 200:    
    players = json.loads(response.text)
    print(json.dumps(players, indent=4))
    save_json(players, 'raw_players.json')

{
    "tier": "CHALLENGER",
    "leagueId": "9a5f071c-c899-3750-b338-7b4eee40f8c0",
    "queue": "RANKED_SOLO_5x5",
    "name": "Lulu's Battlemasters",
    "entries": [
        {
            "summonerId": "y6m2rFj48_6IDcv6ClD5xcSRRmrWEMo38AuuewZDxnCM-UxD",
            "summonerName": "MrCreed",
            "leaguePoints": 615,
            "rank": "I",
            "wins": 281,
            "losses": 250,
            "veteran": false,
            "inactive": false,
            "freshBlood": true,
            "hotStreak": false
        },
        {
            "summonerId": "K4pHoUYKj-ZwYgaUJJ-0digG19IAmA9eoXdC90yjn4yegH4",
            "summonerName": "MRS Kasing",
            "leaguePoints": 639,
            "rank": "I",
            "wins": 182,
            "losses": 156,
            "veteran": false,
            "inactive": false,
            "freshBlood": true,
            "hotStreak": true
        },
        {
            "summonerId": "tvtnccrNk_LjqcOYHhKZ5DBqBloHqkYFu2XmLVfFGKHzi94",

In [3]:
fields = {'summonerId', 'summonerName'}
summoners = [{k: player[k] for k in fields} for player in players['entries']]
print(json.dumps(summoners, indent=4))
save_json(summoners, 'players.json')

[
    {
        "summonerId": "y6m2rFj48_6IDcv6ClD5xcSRRmrWEMo38AuuewZDxnCM-UxD",
        "summonerName": "MrCreed"
    },
    {
        "summonerId": "K4pHoUYKj-ZwYgaUJJ-0digG19IAmA9eoXdC90yjn4yegH4",
        "summonerName": "MRS Kasing"
    },
    {
        "summonerId": "tvtnccrNk_LjqcOYHhKZ5DBqBloHqkYFu2XmLVfFGKHzi94",
        "summonerName": "Back To December"
    },
    {
        "summonerId": "GAqtzyOlT_7RHbpOVd7UD2qZp287yIYNzVz-5TB1ur_Z32E",
        "summonerName": "Envy Carry"
    },
    {
        "summonerId": "z4LgBL_cQT5Nhcq3yTJCYjSIFOFHWq_ivGZ9I7OdMl8Ip2E",
        "summonerName": "Tazaku"
    },
    {
        "summonerId": "DwLHY7nqxGmYVeACoj3I_h-Yt-TEYns74TKV6aSiRDd65FcG",
        "summonerName": "Jeyrus"
    },
    {
        "summonerId": "RDvB5HBZ-ZBUDNdjV3FF6BYv5IpGWj04RT6fCsiKc-7G51A",
        "summonerName": "STARofSCREEN"
    },
    {
        "summonerId": "ET-XdHCNHxgQ-SDF0QX29OVrf6d2mWppz8esnouDQMZhghZl",
        "summonerName": "Carry fanx"
    },
    {
        